## TensorFlow MNIST问题实践优化

选择环境：Anaconda Python 3.5.2  
安装Tensorflow：Python 3.5环境下运行pip install --upgrade --ignore-installed tensorflow  
参考书籍：《TensorFlow实战Google深度学习框架（第2版）》  
ipynb格式：点击阅读原文github

### 5.5 TensorFlow最佳实践样例程序

解决MNIST问题的重构代码拆成3个程序：  
mnist_inference.py：定义前向传播过程以及神经网络中的参数  
mnist_train.py：定义神经网络的训练过程  
mnist_eval.py：定义测试过程

#### 1. mnist_inference.py

In [1]:
import tensorflow as tf

1. 定义神经网络结构相关的参数。

In [2]:
INPUT_NODE = 784
OUTPUT_NODE = 10
LAYER1_NODE = 500

2. 通过tf.get_variable函数来获取变量。

In [4]:
def get_weight_variable(shape, regularizer):
    weights = tf.get_variable(
        "weights", shape, 
        initializer=tf.truncated_normal_initializer(stddev=0.1))
    if regularizer != None: 
        tf.add_to_collection('losses', regularizer(weights))
    return weights

3. 定义神经网络的前向传播过程。

In [5]:
def inference(input_tensor, regularizer):
    # 第一层神经网络及前向传播
    with tf.variable_scope('layer1'):
        weights = get_weight_variable(
            [INPUT_NODE, LAYER1_NODE], regularizer)
        biases = tf.get_variable(
            "biases", [LAYER1_NODE], 
            initializer=tf.constant_initializer(0.0))
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights) + biases)
        
    # 第二层神经网络及前向传播
    with tf.variable_scope('layer2'):
        weights = get_weight_variable(
            [LAYER1_NODE, OUTPUT_NODE], regularizer)
        biases = tf.get_variable(
            "biases", [OUTPUT_NODE], 
            initializer=tf.constant_initializer(0.0))
        layer2 = tf.matmul(layer1, weights) + biases
    
    #返回最后前向传播的结果
    return layer2

无论是训练时还是测试时，都可以直接调用inference这个函数，而不用关心具体的神经网络结构。

#### 2. mnist_train.py

In [6]:
import os
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import mnist_inference

1. 配置神经网络结构相关参数。

In [7]:
BATCH_SIZE = 100 
LEARNING_RATE_BASE = 0.8
LEARNING_RATE_DECAY = 0.99
REGULARIZATION_RATE = 0.0001
TRAINING_STEPS = 20000
MOVING_AVERAGE_DECAY = 0.99 
# 模型保存路径和文件名
MODEL_SAVE_PATH = "C:/Users/74575/Desktop/pythonfile/MNIST_model/"
MODEL_NAME = "mnist_model.ckpt"

2. 定义训练过程，支持程序关闭后从checkpoint恢复训练。

In [8]:
def train(mnist):
    # 定义checkpoint保存点
    ckpt = tf.train.get_checkpoint_state(MODEL_SAVE_PATH)
    # 定义输入输出placeholder。
    x = tf.placeholder(
        tf.float32, [None, mnist_inference.INPUT_NODE], name='x-input')
    y_ = tf.placeholder(
        tf.float32, [None, mnist_inference.OUTPUT_NODE], name='y-input')

    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    # 直接使用mnist_inference.py中定义的前向传播过程
    y = mnist_inference.inference(x, regularizer)
    global_step = tf.Variable(0, trainable=False)
    
    # 定义损失函数、学习率、滑动平均操作以及训练过程。
    variable_averages = tf.train.ExponentialMovingAverage(
        MOVING_AVERAGE_DECAY, global_step)
    variables_averages_op = variable_averages.apply(
        tf.trainable_variables())
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=y, labels=tf.argmax(y_, 1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    loss = cross_entropy_mean + tf.add_n(tf.get_collection('losses'))
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,
        global_step,
        mnist.train.num_examples / BATCH_SIZE, 
        LEARNING_RATE_DECAY,
        staircase=True)
    train_step = tf.train.GradientDescentOptimizer(learning_rate)\
                   .minimize(loss, global_step=global_step)
    with tf.control_dependencies([train_step, variables_averages_op]):
        train_op = tf.no_op(name='train')
        
    # 初始化TensorFlow持久化类。
    saver = tf.train.Saver()  
    with tf.Session() as sess:
        saved_step = 0
        if ckpt and ckpt.model_checkpoint_path:
            print("checkpoint存在，直接恢复变量")
            saver.restore(sess, ckpt.model_checkpoint_path)
            # 恢复global_step
            saved_step = int(ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1])
            sess.run(global_step.assign(saved_step))
        else:
            print("checkpoint不存在，进行变量初始化")
            tf.global_variables_initializer().run()

        for i in range(saved_step, TRAINING_STEPS):
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            _, loss_value, step = sess.run([train_op, loss, global_step], feed_dict={x: xs, y_: ys})
            if i % 1000 == 0:
                print("After %d training step(s), loss on training batch is %g." % (step, loss_value))
                saver.save(sess, os.path.join(MODEL_SAVE_PATH, MODEL_NAME), global_step=global_step)
                last_step = step - 1000
                if last_step > 0:
                    try:
                        os.remove(MODEL_SAVE_PATH+MODEL_NAME+"-"+str(last_step)+".index")
                        os.remove(MODEL_SAVE_PATH+MODEL_NAME+"-"+str(last_step)+".data-00000-of-00001")
                        os.remove(MODEL_SAVE_PATH+MODEL_NAME+"-"+str(last_step)+".meta")
                    except:
                        print("删除数据异常")
                    else:
                        print("成功删除：", MODEL_SAVE_PATH+MODEL_NAME+"-"+str(last_step)+".*")

3. 主程序入口

In [9]:
tf.reset_default_graph() # 这玩意儿很重要！
def main(argv=None):
    mnist = input_data.read_data_sets("../../datasets/MNIST_data", one_hot=True)
    train(mnist)

if __name__ == '__main__':
    main()

Extracting ../../datasets/MNIST_data\train-images-idx3-ubyte.gz
Extracting ../../datasets/MNIST_data\train-labels-idx1-ubyte.gz
Extracting ../../datasets/MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ../../datasets/MNIST_data\t10k-labels-idx1-ubyte.gz
checkpoint不存在，进行变量初始化
After 1 training step(s), loss on training batch is 3.01294.
After 1001 training step(s), loss on training batch is 0.281872.
成功删除： C:/Users/74575/Desktop/pythonfile/MNIST_model/mnist_model.ckpt-1.*
After 2001 training step(s), loss on training batch is 0.171331.
成功删除： C:/Users/74575/Desktop/pythonfile/MNIST_model/mnist_model.ckpt-1001.*
After 3001 training step(s), loss on training batch is 0.143659.
成功删除： C:/Users/74575/Desktop/pythonfile/MNIST_model/mnist_model.ckpt-2001.*
After 4001 training step(s), loss on training batch is 0.112762.
成功删除： C:/Users/74575/Desktop/pythonfile/MNIST_model/mnist_model.ckpt-3001.*
After 5001 training step(s), loss on training batch is 0.103214.
成功删除： C:/Users/74575/Desktop/pythonfi

#### 3. mnist_eval.py

In [1]:
import time
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import mnist_inference
import mnist_train

1. 每10秒加载一次最新的模型

In [2]:
# 加载的时间间隔。
EVAL_INTERVAL_SECS = 10

def evaluate(mnist):
    with tf.Graph().as_default() as g:
        # 定义输入输出的格式
        x = tf.placeholder(
            tf.float32, [None, mnist_inference.INPUT_NODE], name='x-input')
        y_ = tf.placeholder(
            tf.float32, [None, mnist_inference.OUTPUT_NODE], name='y-input')
        validate_feed = {x: mnist.validation.images, 
                         y_: mnist.validation.labels}

        y = mnist_inference.inference(x, None)
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

        variable_averages = tf.train.ExponentialMovingAverage(mnist_train.MOVING_AVERAGE_DECAY)
        variables_to_restore = variable_averages.variables_to_restore()
        saver = tf.train.Saver(variables_to_restore)

        while True:
            with tf.Session() as sess:
                ckpt = tf.train.get_checkpoint_state(mnist_train.MODEL_SAVE_PATH)
                if ckpt and ckpt.model_checkpoint_path:
                    saver.restore(sess, ckpt.model_checkpoint_path)
                    for v in tf.global_variables():
                        print(v.name, ":", v.eval())
                    print("#####################")    
                    global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
                    accuracy_score = sess.run(accuracy, feed_dict=validate_feed)
                    print("After %s training step(s), validation accuracy = %g" % (global_step, accuracy_score))
                else:
                    print('No checkpoint file found')
                    return
            time.sleep(EVAL_INTERVAL_SECS)

##### 主程序

In [ ]:
tf.reset_default_graph()
def main(argv=None):
    mnist = input_data.read_data_sets("../../datasets/MNIST_data", one_hot=True)
    evaluate(mnist)

if __name__ == '__main__':
    main()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../datasets/MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../datasets/MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../../datasets/MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ../../datasets/MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
INFO:tensorflow:Restoring parameters from C:/Users/74575/Desktop/pythonfile/MNIST_model/mnist_model.ckpt-19001
layer1/weights:0 : [[-0.01733263 -0.03433179 -0.02238548 ... -0.00208434  0.04237024
   0.04845152]
 [ 0.02311971 -0.00975731 -0.0

INFO:tensorflow:Restoring parameters from C:/Users/74575/Desktop/pythonfile/MNIST_model/mnist_model.ckpt-19001
layer1/weights:0 : [[-0.01733263 -0.03433179 -0.02238548 ... -0.00208434  0.04237024
   0.04845152]
 [ 0.02311971 -0.00975731 -0.00799027 ... -0.04326474  0.01992522
  -0.00845015]
 [-0.00522756 -0.00932694 -0.01814423 ... -0.03246705  0.01844428
  -0.02531365]
 ...
 [-0.01022352 -0.00611304 -0.00699913 ... -0.03299887 -0.00392862
   0.01065448]
 [ 0.00821234 -0.02065444  0.00473766 ...  0.03866515  0.01196826
  -0.02328382]
 [ 0.05252424  0.02343992  0.00955723 ...  0.01668027  0.01626522
  -0.00162148]]
layer1/biases:0 : [ 8.56852392e-04 -3.21874350e-01  9.93898371e-04 -2.40885243e-01
 -1.92076748e-03 -6.13231845e-02 -2.02678740e-01  5.41952737e-02
 -2.16180715e-03 -5.01906797e-02 -1.64470106e-01 -2.40195602e-01
 -1.13565877e-01  1.15982285e-02 -1.07233159e-01 -4.54190373e-02
  2.57977452e-02  1.16784899e-02  1.99265271e-01  4.45566922e-02
 -6.18233010e-02  2.80489624e-01  1

INFO:tensorflow:Restoring parameters from C:/Users/74575/Desktop/pythonfile/MNIST_model/mnist_model.ckpt-19001
layer1/weights:0 : [[-0.01733263 -0.03433179 -0.02238548 ... -0.00208434  0.04237024
   0.04845152]
 [ 0.02311971 -0.00975731 -0.00799027 ... -0.04326474  0.01992522
  -0.00845015]
 [-0.00522756 -0.00932694 -0.01814423 ... -0.03246705  0.01844428
  -0.02531365]
 ...
 [-0.01022352 -0.00611304 -0.00699913 ... -0.03299887 -0.00392862
   0.01065448]
 [ 0.00821234 -0.02065444  0.00473766 ...  0.03866515  0.01196826
  -0.02328382]
 [ 0.05252424  0.02343992  0.00955723 ...  0.01668027  0.01626522
  -0.00162148]]
layer1/biases:0 : [ 8.56852392e-04 -3.21874350e-01  9.93898371e-04 -2.40885243e-01
 -1.92076748e-03 -6.13231845e-02 -2.02678740e-01  5.41952737e-02
 -2.16180715e-03 -5.01906797e-02 -1.64470106e-01 -2.40195602e-01
 -1.13565877e-01  1.15982285e-02 -1.07233159e-01 -4.54190373e-02
  2.57977452e-02  1.16784899e-02  1.99265271e-01  4.45566922e-02
 -6.18233010e-02  2.80489624e-01  1